In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Endometrioid_Cancer"
cohort = "GSE73614"

# Input paths
in_trait_dir = "../../input/GEO/Endometrioid_Cancer"
in_cohort_dir = "../../input/GEO/Endometrioid_Cancer/GSE73614"

# Output paths
out_data_file = "../../output/preprocess/Endometrioid_Cancer/GSE73614.csv"
out_gene_data_file = "../../output/preprocess/Endometrioid_Cancer/gene_data/GSE73614.csv"
out_clinical_data_file = "../../output/preprocess/Endometrioid_Cancer/clinical_data/GSE73614.csv"
json_path = "../../output/preprocess/Endometrioid_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Can High Grade Serous Ovarian Cancer TCGA Gene Expression Signatures be seen in High Grade Endometrioid or Clear Cell Ovarian Cancer?"
!Series_summary	"The goal of the study was to examine the transcriptional profile of ovarian cancer cancers in order to develop validated clinically useful prognostic signatures with the potential to guide therapy decisions.  Fresh frozen samples were prospectively collected from a series of 107 consecutive women with high-grade serous ovarian, primary peritonial, or fallopian tube cancer as well as high grade clear cell and endometrioid cancer who underwent surgery by a gynecologic oncologist at Mayo Clinic between 1994 and 2005. All patients received postoperative chemotherapy with a platinum agent, and 75% received a taxane. All patients signed an Institutional Review Board approved consent for bio-banking, clinical data extraction and molecular analysis. Median follow-up time was 35 months (range, 1-202 months)

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import re
import pandas as pd
import numpy as np
import json
import os
from typing import Optional, Callable, Dict, Any, List, Union, Tuple

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains transcriptional profiles
# of ovarian cancers, suggesting gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# From the sample characteristics dictionary, we only have 'tissue: ovarian'
# and no information about the trait (Endometrioid_Cancer), age, or gender

# 2.1 Data Availability
# The sample characteristics don't explicitly mention Endometrioid Cancer
# However, the background info mentions "high grade clear cell and endometrioid cancer"
# Since the dataset includes endometrioid cancer samples, we can attempt to identify these from other data
# But given the current information, we don't have a specific row that indicates trait status
trait_row = None
age_row = None
gender_row = None

# 2.2 Data Type Conversion
# Even though we don't have direct trait information, we'll define conversion functions
# in case we can infer trait status from other data later
def convert_trait(value):
    if not value or pd.isna(value):
        return None
    
    # Extract value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Check for endometrioid cancer indicators
    lower_value = str(value).lower()
    if 'endometrioid' in lower_value:
        return 1
    elif 'not endometrioid' in lower_value or 'serous' in lower_value or 'clear cell' in lower_value:
        return 0
    else:
        return None

def convert_age(value):
    if not value or pd.isna(value):
        return None
    
    # Extract value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Try to extract age as a number
    try:
        # Extract numbers from the string
        numbers = re.findall(r'\d+', str(value))
        if numbers:
            return float(numbers[0])
    except:
        pass
    
    return None

def convert_gender(value):
    if not value or pd.isna(value):
        return None
    
    # Extract value after colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert gender values to binary (0 for female, 1 for male)
    lower_value = str(value).lower()
    if 'female' in lower_value or 'f' == lower_value:
        return 0
    elif 'male' in lower_value or 'm' == lower_value:
        return 1
    else:
        return None

# 3. Save Metadata
# Since trait_row is None, is_trait_available is False
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip this substep


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 88
Header line: "ID_REF"	"GSM1899433"	"GSM1899434"	"GSM1899435"	"GSM1899436"	"GSM1899437"	"GSM1899438"	"GSM1899439"	"GSM1899440"	"GSM1899441"	"GSM1899442"	"GSM1899443"	"GSM1899444"	"GSM1899445"	"GSM1899446"	"GSM1899447"	"GSM1899448"	"GSM1899449"	"GSM1899450"	"GSM1899451"	"GSM1899452"	"GSM1899453"	"GSM1899454"	"GSM1899455"	"GSM1899456"	"GSM1899457"	"GSM1899458"	"GSM1899459"	"GSM1899460"	"GSM1899461"	"GSM1899462"	"GSM1899463"	"GSM1899464"	"GSM1899465"	"GSM1899466"	"GSM1899467"	"GSM1899468"	"GSM1899469"	"GSM1899470"	"GSM1899471"	"GSM1899472"	"GSM1899473"	"GSM1899474"	"GSM1899475"	"GSM1899476"	"GSM1899477"	"GSM1899478"	"GSM1899479"	"GSM1899480"	"GSM1899481"	"GSM1899482"	"GSM1899483"	"GSM1899484"	"GSM1899485"	"GSM1899486"	"GSM1899487"	"GSM1899488"	"GSM1899489"	"GSM1899490"	"GSM1899491"	"GSM1899492"	"GSM1899493"	"GSM1899494"	"GSM1899495"	"GSM1899496"	"GSM1899497"	"GSM1899498"	"GSM1899499"	"GSM1899500"	"GSM1899501"	"GSM1899502"	"GSM1899503"	"GSM1899504"	"GSM1899505"	

Index(['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056',
       'A_23_P100074', 'A_23_P100092', 'A_23_P100103', 'A_23_P100111',
       'A_23_P100127', 'A_23_P100133', 'A_23_P100141', 'A_23_P100156',
       'A_23_P100177', 'A_23_P100189', 'A_23_P100196', 'A_23_P100203',
       'A_23_P100220', 'A_23_P100240', 'A_23_P10025', 'A_23_P100263'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers from the previous step, these are Agilent microarray probe identifiers
# (format A_23_P######) rather than human gene symbols.
# These will need to be mapped to gene symbols for biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE73614
Line 6: !Series_title = Can High Grade Serous Ovarian Cancer TCGA Gene Expression Signatures be seen in High Grade Endometrioid or Clear Cell Ovarian Cancer?
Line 7: !Series_geo_accession = GSE73614
Line 8: !Series_status = Public on Oct 01 2015
Line 9: !Series_submission_date = Sep 30 2015
Line 10: !Series_last_update_date = Oct 07 2019
Line 11: !Series_pubmed_id = 27016234
Line 12: !Series_summary = The goal of the study was to examine the transcriptional profile of ovarian cancer cancers in order to develop validated clinically useful prognostic signatures with the potential to guide therapy decisions.  Fresh frozen samples were prospectively collected from a series of 107 consecutive women with


Gene annotation preview:
{'ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'SPOT_ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'CONTROL_TYPE': ['FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GB_ACC': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GENE': [400451.0, 10239.0, 9899.0, 348093.0, 57099.0], 'GENE_SYMBOL': ['FAM174B', 'AP3S2', 'SV2B', 'RBPMS2', 'AVEN'], 'GENE_NAME': ['family with sequence similarity 174, member B', 'adaptor-related protein complex 3, sigma 2 subunit', 'synaptic vesicle glycoprotein 2B', 'RNA binding protein with multiple splicing 2', 'apoptosis, caspase activation inhibitor'], 'UNIGENE_ID': ['Hs.27373', 'Hs.632161', 'Hs.21754', 'Hs.436518', 'Hs.555966'], 'ENSEMBL_ID': ['ENST00000557398', nan, 'ENST00000557410', 'ENST00000300069', 'ENST00000306730'], 'TIGR_ID': [nan, nan, nan, nan, nan

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns for mapping
# From the gene_annotation preview, we can see:
# - 'ID' column contains probe identifiers (A_23_P######) which match the gene expression data's indices
# - 'GENE_SYMBOL' column contains the human gene symbols we want to map to

# 2. Create the gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')

# 3. Apply gene mapping to convert probe-level measurements to gene expression
# This handles the many-to-many mapping by dividing expression values when one probe maps
# to multiple genes, then summing all probe contributions for each gene
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the gene expression data after mapping
print("Gene expression data after mapping to gene symbols:")
print(f"Shape: {gene_data.shape}")
print(f"First few gene symbols: {gene_data.index[:10]}")


Gene expression data after mapping to gene symbols:
Shape: (18488, 107)
First few gene symbols: Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT',
       'AAAS', 'AACS'],
      dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print("First few genes with their expression values after normalization:")
print(normalized_gene_data.head())

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if trait data is available before proceeding with clinical data extraction
if trait_row is None:
    print("Trait row is None. Cannot extract trait information from clinical data.")
    # Create an empty dataframe for clinical features
    clinical_features = pd.DataFrame()
    
    # Create an empty dataframe for linked data
    linked_data = pd.DataFrame()
    
    # Validate and save cohort info
    validate_and_save_cohort_info(
        is_final=True, 
        cohort=cohort, 
        info_path=json_path, 
        is_gene_available=True, 
        is_trait_available=False,  # Trait data is not available
        is_biased=True,  # Not applicable but required
        df=pd.DataFrame(),  # Empty dataframe
        note="Dataset contains gene expression data but lacks clear trait indicators for Duchenne Muscular Dystrophy status."
    )
    print("Data was determined to be unusable due to missing trait indicators and was not saved")
else:
    try:
        # Get the file paths for the matrix file to extract clinical data
        _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
        
        # Get raw clinical data from the matrix file
        _, clinical_raw = get_background_and_clinical_data(matrix_file)
        
        # Verify clinical data structure
        print("Raw clinical data shape:", clinical_raw.shape)
        
        # Extract clinical features using the defined conversion functions
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_raw,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        print("Clinical features:")
        print(clinical_features)
        
        # Save clinical features to file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
        
        # 3. Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        print("Linked data preview (first 5 rows, first 5 columns):")
        print(linked_data.iloc[:5, :5])
        
        # 4. Handle missing values
        print("Missing values before handling:")
        print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
        if 'Age' in linked_data.columns:
            print(f"  Age missing: {linked_data['Age'].isna().sum()} out of {len(linked_data)}")
        if 'Gender' in linked_data.columns:
            print(f"  Gender missing: {linked_data['Gender'].isna().sum()} out of {len(linked_data)}")
        
        gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
        print(f"  Genes with >20% missing: {sum(linked_data[gene_cols].isna().mean() > 0.2)}")
        print(f"  Samples with >5% missing genes: {sum(linked_data[gene_cols].isna().mean(axis=1) > 0.05)}")
        
        cleaned_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {cleaned_data.shape}")
        
        # 5. Evaluate bias in trait and demographic features
        is_trait_biased = False
        if len(cleaned_data) > 0:
            trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
            is_trait_biased = trait_biased
        else:
            print("No data remains after handling missing values.")
            is_trait_biased = True
        
        # 6. Final validation and save
        is_usable = validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=True, 
            is_biased=is_trait_biased, 
            df=cleaned_data,
            note="Dataset contains gene expression data comparing Duchenne muscular dystrophy vs healthy samples."
        )
        
        # 7. Save if usable
        if is_usable and len(cleaned_data) > 0:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            cleaned_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Data was determined to be unusable or empty and was not saved")
            
    except Exception as e:
        print(f"Error processing data: {e}")
        # Handle the error case by still recording cohort info
        validate_and_save_cohort_info(
            is_final=True, 
            cohort=cohort, 
            info_path=json_path, 
            is_gene_available=True, 
            is_trait_available=False,  # Mark as not available due to processing issues
            is_biased=True, 
            df=pd.DataFrame(),  # Empty dataframe
            note=f"Error processing data: {str(e)}"
        )
        print("Data was determined to be unusable and was not saved")

Normalized gene data shape: (18247, 107)
First few genes with their expression values after normalization:
          GSM1899433  GSM1899434  GSM1899435  GSM1899436  GSM1899437  \
Gene                                                                   
A1BG        -0.38515     0.02804     0.22406     0.06810    -0.41163   
A1BG-AS1    -0.14558     0.09419     0.22782     0.04897    -0.18101   
A1CF        -0.05427     0.00154    -0.08773    -0.04492    -0.00743   
A2M         -0.29231    -0.14008     0.01138    -0.24503    -0.57375   
A2ML1        0.27110     0.31217     0.21735     0.26602     0.21648   

          GSM1899438  GSM1899439  GSM1899440  GSM1899441  GSM1899442  ...  \
Gene                                                                  ...   
A1BG        -0.04018     0.22088     0.04537     0.16108     0.12155  ...   
A1BG-AS1    -0.02011     0.18320     0.24031     0.11782    -0.02379  ...   
A1CF        -0.04886    -0.00683    -0.02361    -0.10952     0.09634  ...   
A2M

Normalized gene data saved to ../../output/preprocess/Endometrioid_Cancer/gene_data/GSE73614.csv
Trait row is None. Cannot extract trait information from clinical data.
Abnormality detected in the cohort: GSE73614. Preprocessing failed.
Data was determined to be unusable due to missing trait indicators and was not saved
